In [1]:
# %pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


**Visualización de documentos**

In [6]:
for idx in range(100, 110):
    print(f"Documento {idx}:\n")
    print(newsgroups_train.data[idx])
    print("-" * 80)

Documento 100:

1.  Software publishing SuperBase 4 windows v.1.3           --->$80

2.  OCR System ReadRight v.3.1 for Windows                  --->$65

3.  OCR System ReadRight  v.2.01 for DOS                    --->$65

4.  Unregistered Zortech 32 bit C++ Compiler v.3.1          --->$ 250
     with Multiscope windows Debugger,
     WhiteWater Resource Toolkit, Library Source Code

5.  Glockenspiel/ImageSoft Commonview 2 Windows
     Applications Framework for Borland C++                 --->$70

6.  Spontaneous Assembly Library With Source Code           --->$50

7.  Microsoft Macro Assembly 6.0                            --->$50

8.  Microsoft Windows v.3.1 SDK Documentation               --->$125

9.  Microsoft FoxPro V.2.0                                  --->$75

10.  WordPerfect 5.0 Developer's Toolkit                    --->$20

11.  Kedwell Software DataBoss v.3.5 C Code Generator       --->$100

12.  Kedwell InstallBoss v.2.0 Installation Generator       --->$35

13.  Liant 

In [7]:
print(newsgroups_train.target[0])

7


In [8]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

Cada documento es un vector de 101631 entradas cada uno donde
hay ceros por las palabras que no aparecen y números por las que sí.


In [9]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


- Diccionario opuesto

In [10]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [11]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [12]:
idx2word[25775]

'car'

In [13]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [14]:
len(np.unique(y_train))

20

In [15]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [16]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [17]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [18]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [19]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [20]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [21]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [22]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


Existe una _relación_ entre la similitud del coseno de los documentos y las categorias a las que pertenecen.

### Modelo de clasificación Naïve Bayes

In [23]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [24]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [25]:
X_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 665738 stored elements and shape (7532, 101631)>

In [26]:
y_test

array([ 7,  5,  0, ...,  9,  6, 15])

In [27]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

---

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.



**Lista de ids**

In [28]:
idx_list = [10995, 9010, 9009, 8101, 11300]

**Similaridad**

In [29]:
results = {}

for idx in idx_list:
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    sorted_indices = np.argsort(cossim)[::-1]  # sort

    most_similar_idxs = sorted_indices[1:6]
    most_similar_classes = [newsgroups_train.target_names[y_train[i]] for i in most_similar_idxs]

    results[idx] = {
        "top_similarities": np.sort(cossim)[::-1][1:6],
        "top_documents": most_similar_idxs,
        "top_classes": most_similar_classes,
        "original_class": newsgroups_train.target_names[y_train[idx]]
    }

for idx, data in results.items():
    print(f"ID {idx}:")
    print(f" - Clase original: {data['original_class']}")
    print(f" - Documentos más similares: {data['top_documents']}")
    print(f" - Valores de similaridad: {data['top_similarities']}")
    print(f" - Clases de los documentos más similares: {data['top_classes']}\n")


ID 10995:
 - Clase original: soc.religion.christian
 - Documentos más similares: [8714 5559 5826  913 9623]
 - Valores de similaridad: [0.59848939 0.50043473 0.49200999 0.48865432 0.48402305]
 - Clases de los documentos más similares: ['soc.religion.christian', 'soc.religion.christian', 'soc.religion.christian', 'alt.atheism', 'talk.politics.mideast']

ID 9010:
 - Clase original: sci.electronics
 - Documentos más similares: [ 5279  9623  8754  9192 10575]
 - Valores de similaridad: [0.80126885 0.32648548 0.31034654 0.30968585 0.30870909]
 - Clases de los documentos más similares: ['sci.electronics', 'talk.politics.mideast', 'talk.religion.misc', 'sci.electronics', 'sci.crypt']

ID 9009:
 - Clase original: comp.windows.x
 - Documentos más similares: [ 627 1118 1702 5979 6051]
 - Valores de similaridad: [0.75489157 0.67888784 0.49863827 0.45889587 0.34259741]
 - Clases de los documentos más similares: ['comp.windows.x', 'comp.windows.x', 'comp.windows.x', 'comp.windows.x', 'comp.windows.

- Análisis por contenido de texto

In [30]:
# results.items()

In [31]:
print(f"Contenido del documento {idx}:\n", newsgroups_train.data[idx])

Contenido del documento 11300:
 
I am asking you to believe in things not visible. I don't know if this is
believeing blindly or not. I'm not sure how blindness comes into it. I do
not deny reason, indeed I insist upon it, but reason only draws conclusions
from evidence. If you decide in advance that your reason will act only on
the evidence of the five physical senses, then you cut reason off from any
possibility of reaching a conclusion outside the physical sphere (beyond the
rather provocative, if inconclusive, conclusion that the physical sphere
is not self explanatory). 

Christians claim that they have received a different kind of evidence, 
which they call faith, and which is a gift of God. That is, this evidence
is the evidence of a thing which chooses to reveal or hide itself. The 
evidence of the senses cannot tell you is such a ting exists. Reasoning
on the evidence of the senses won't help either. But Christians do reason
of the evidence of faith, and do claim that this evi

In [32]:
for doc_id in data['top_documents']:
    print(f"\nContenido del documento {doc_id}:\n")
    print(newsgroups_train.data[doc_id])
    print("-" * 80)


Contenido del documento 11206:


believeing blindly or not. I'm not sure how blindness comes into it. I do > not
deny reason, indeed I insist upon it, but reason only draws conclusions > from
evidence. If you decide in advance that your reason will act only on > the
evidence of the five physical senses, then you cut reason off from any >
possibility of reaching a conclusion outside the physical sphere (beyond the >
rather provocative, if inconclusive, conclusion that the physical sphere > is
not self explanatory). 

So your are saying to rely on our feelings and experiences (since
this is the only other source of information left to us).
How can you then convince somebody that your "feelings and experiences"
are the correct ones then if you can't show somebody visible and
measurable effects?  If my experiences say that "there exists no god"
and yours says there does, where does that leave us?  Since we are only
going on experiences, then both of us are correct within our own personal



**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**Distribución de clases**

In [33]:
class_counts = np.bincount(newsgroups_test.target)
sorted_classes = sorted(zip(class_counts, newsgroups_test.target_names), reverse=True, key=lambda x: x[0])

for count, class_name in sorted_classes:
    print(f"'{class_name}': {count}")

'rec.sport.hockey': 399
'rec.motorcycles': 398
'soc.religion.christian': 398
'rec.sport.baseball': 397
'rec.autos': 396
'sci.crypt': 396
'sci.med': 396
'comp.windows.x': 395
'comp.os.ms-windows.misc': 394
'sci.space': 394
'sci.electronics': 393
'comp.sys.ibm.pc.hardware': 392
'misc.forsale': 390
'comp.graphics': 389
'comp.sys.mac.hardware': 385
'talk.politics.mideast': 376
'talk.politics.guns': 364
'alt.atheism': 319
'talk.politics.misc': 310
'talk.religion.misc': 251


In [34]:
tfidfvect = TfidfVectorizer(strip_accents='unicode',
                            stop_words='english',
                            lowercase=True
                            )
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [35]:
models = [MultinomialNB(alpha=0.5), ComplementNB(alpha=0.2)]

for model in models:

    model.fit(X_train, y_train)

    # Transform test
    X_test = tfidfvect.transform(newsgroups_test.data)
    y_test = newsgroups_test.target

    y_pred = model.predict(X_test)

    score = f1_score(y_test, y_pred, average='macro')
    print(f"{model.__class__.__name__} - F1 Score: {score:.4f}")


MultinomialNB - F1 Score: 0.6584
ComplementNB - F1 Score: 0.6972


> Para este ejercicio se probaron los parámetros `strip_accents`, `stop_words`, `lowercase` y el parámetro `alpha` en los modelos, esto implicó el aumento de la métrica f1 score de 0.58 a 0.65 aproximadamente.
El modelo ComplementNB presenta una leve mejoría respecto del MultinomialNB, esto podría darse porque el mismo es particularmente para datasets desbalanceados.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


Matriz documento-término

In [36]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 755809 stored elements and shape (11314, 101321)>
  Coords	Values
  (0, 96880)	0.1370359812611726
  (0, 37257)	0.20599311323287348
  (0, 25718)	0.46579831435138963
  (0, 80421)	0.12706903967122096
  (0, 31928)	0.10526008886822913
  (0, 34742)	0.14847880131844232
  (0, 84313)	0.1636839250592851
  (0, 57248)	0.13520842471059058
  (0, 55607)	0.1382259698975382
  (0, 9844)	0.20797700857530219
  (0, 35903)	0.12667096041973439
  (0, 11175)	0.20599311323287348
  (0, 25438)	0.10548299054214268
  (0, 24109)	0.2472313451421643
  (0, 34743)	0.17300821242559042
  (0, 76270)	0.08978258481915571
  (0, 83209)	0.11339406589538421
  (0, 16807)	0.14077745547061019
  (0, 24584)	0.19644480500804057
  (0, 81451)	0.14613089342888969
  (0, 77677)	0.12197186951739483
  (0, 23431)	0.1293710328851233
  (0, 54495)	0.06961997844491916
  (0, 87914)	0.25808578247347563
  (0, 62595)	0.13037295035007845
  :	:
  (11313, 47086)	0.15717057502749704
  (11313, 

Matriz término-documento | Vectorización por palabras

In [37]:
X_train_T = X_train.T
print(X_train_T.shape, '\n', type(X_train_T))

(101321, 11314) 
 <class 'scipy.sparse._csc.csc_matrix'>


In [38]:
feature_n = tfidfvect.get_feature_names_out()
words_list = ["automatic","data","analysis","events","furthermore"]
# words idx
word_indices = [np.where(feature_n == word)[0][0] for word in words_list]

In [39]:
results = {}

for idx, word in zip(word_indices, words_list):
    cossim = cosine_similarity(X_train_T[idx], X_train_T)[0]
    sorted_indices = np.argsort(cossim)[::-1][1:6]

    #most_similar_idxs = sorted_indices[1:6]
    most_similar_terms = [feature_n[i] for i in sorted_indices]

    results[word] = {
        "top_similarities": np.sort(cossim)[::-1][1:6],
        "top_terms": most_similar_idxs,
        "top_terms_names": most_similar_terms,
    }

for word, data in results.items():
    print(f"Palabra: '{word}'")
    print(f" - Términos más similares: {data['top_terms_names']}")
    print(f" - Índices de los términos más similares: {data['top_terms']}")
    print(f" - Valores de similaridad: {data['top_similarities']}\n")


Palabra: 'automatic'
 - Términos más similares: ['chore', 'disabilities', 'automatics', 'transmissions', 'transmission']
 - Índices de los términos más similares: [11206  6663   468 10836 10052]
 - Valores de similaridad: [0.37858319 0.37738885 0.32771706 0.28211182 0.22820837]

Palabra: 'data'
 - Términos más similares: ['gigerish', 'encapsulating', 'uncompressor', 'bettween', 'landsat']
 - Índices de los términos más similares: [11206  6663   468 10836 10052]
 - Valores de similaridad: [0.19294141 0.19294141 0.19294141 0.1717929  0.16452746]

Palabra: 'analysis'
 - Términos más similares: ['hurdles', 'taggants', 'histograms', 'analyze', 'barbers']
 - Índices de los términos más similares: [11206  6663   468 10836 10052]
 - Valores de similaridad: [0.31730918 0.28099843 0.27137443 0.2311833  0.23091759]

Palabra: 'events'
 - Términos más similares: ['dispatcher', 'queue', 'rescale', 'appl', 'cascading']
 - Índices de los términos más similares: [11206  6663   468 10836 10052]
 - Valor

> Podemos observar que, adquiriendo esa nueva representación (término-documento), se ven relaciones entre palabras que están en contextos similares, sin embargo parece haber ruido en algunas asociaciones de las palabras `data`, `events`, lo que podría mejorarse ajustando parámetros como eliminar terminos irrelevantes.